## 读取数据

In [3]:
import pickle

with open('train_content_list.pickle', 'rb') as file:
    train_content_list = pickle.load(file)
with open('train_label_list.pickle', 'rb') as file:
    train_label_list = pickle.load(file)
with open('test_content_list.pickle', 'rb') as file:
    test_content_list = pickle.load(file)
with open('test_label_list.pickle', 'rb') as file:
    test_label_list = pickle.load(file)

## 制作词汇表

In [4]:
from collections import Counter

def getVocabularyList(content_list, vocabulary_size):
    allContent_str = ''.join(content_list)
    counter = Counter(allContent_str)
    vocabulary_list = [k[0] for k in counter.most_common(vocabulary_size)]
    return ['PAD'] + vocabulary_list 

def makeVocabularyFile(content_list, vocabulary_size):
    vocabulary_list = getVocabularyList(content_list, vocabulary_size)
    with open('vocabulary.txt', 'w', encoding='utf8') as file:
        for vocabulary in vocabulary_list:
            file.write(vocabulary + '\n')

makeVocabularyFile(train_content_list, 2000)

#### 读取词汇表

In [5]:
with open('vocabulary.txt', encoding='utf8') as file:
    vocabulary_list = [k.strip() for k in file.readlines()]

## 数据准备

In [6]:
word2id_dict = dict([(b, a) for a, b in enumerate(vocabulary_list)])
content2idList = lambda content : [word2id_dict[word] for word in content if word in word2id_dict]
train_idlist_list = [content2idList(content) for content in train_content_list]

In [7]:
vocab_size = 5000 if len(vocabulary_list) > 5000 else len(vocabulary_list)
vocab_size

541

In [8]:
contentLength_list = [len(k) for k in train_content_list]
seq_length = max(contentLength_list)
seq_length

125

In [10]:
embedding_dim = 64  # 词向量维度
num_classes = 4  # 类别数
num_filters = 256  # 卷积核数目
kernel_size = 5  # 卷积核尺
hidden_dim = 128  # 全连接层神经元
dropout_keep_prob = 0.5  # dropout保留比例
learning_rate = 1e-3  # 学习率
batch_size = 32  # 每批训练大小

In [11]:
import tensorflow.contrib.keras as kr
train_X = kr.preprocessing.sequence.pad_sequences(train_idlist_list, seq_length)
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
train_y = labelEncoder.fit_transform(train_label_list)
train_Y = kr.utils.to_categorical(train_y, num_classes)
import tensorflow as tf
tf.reset_default_graph()
X_holder = tf.placeholder(tf.int32, [None, seq_length])
Y_holder = tf.placeholder(tf.float32, [None, num_classes])

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 搭建神经网络

In [14]:
embedding = tf.get_variable('embedding', [vocab_size, embedding_dim])
embedding_inputs = tf.nn.embedding_lookup(embedding, X_holder)
conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size)
max_pooling = tf.reduce_max(conv, reduction_indices=[1])
full_connect = tf.layers.dense(max_pooling, hidden_dim)
full_connect_dropout = tf.contrib.layers.dropout(full_connect, keep_prob=0.75)
full_connect_activate = tf.nn.relu(full_connect_dropout)
softmax_before = tf.layers.dense(full_connect_activate, num_classes)
predict_Y = tf.nn.softmax(softmax_before)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_holder, logits=softmax_before)
loss = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)
isCorrect = tf.equal(tf.argmax(Y_holder, 1), tf.argmax(predict_Y, 1))
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


## 参数初始化

In [20]:
init = tf.global_variables_initializer()
session = tf.Session()
session.run(init)

## 模型训练

In [21]:
import pickle
with open('test_content_list.pickle', 'rb') as file:
    test_content_list = pickle.load(file)
with open('test_label_list.pickle', 'rb') as file:
    test_label_list = pickle.load(file)
test_idlist_list = [content2idList(content) for content in test_content_list]
test_X = kr.preprocessing.sequence.pad_sequences(test_idlist_list, seq_length)
test_y = labelEncoder.transform(test_label_list)
test_Y = kr.utils.to_categorical(test_y, num_classes)

import random
for i in range(100):
    selected_index = random.sample(list(range(len(train_y))), k=batch_size)
    batch_X = train_X[selected_index]
    batch_Y = train_Y[selected_index]
    session.run(train, {X_holder:batch_X, Y_holder:batch_Y})
    step = i + 1 
    if step % 10 == 0 or step == 1:
        selected_index = random.sample(list(range(len(test_y))), k=200)
        batch_X = test_X[selected_index]
        batch_Y = test_Y[selected_index]
        loss_value, accuracy_value = session.run([loss, accuracy], {X_holder:batch_X, Y_holder:batch_Y})
        print('step:%d loss:%.4f accuracy:%.4f' %(step, loss_value, accuracy_value))

step:1 loss:1.3800 accuracy:0.2550
step:10 loss:1.2933 accuracy:0.2250
step:20 loss:0.9269 accuracy:0.8150
step:30 loss:0.4702 accuracy:0.9550
step:40 loss:0.1596 accuracy:0.9950
step:50 loss:0.0567 accuracy:1.0000
step:60 loss:0.0185 accuracy:1.0000
step:70 loss:0.0091 accuracy:1.0000
step:80 loss:0.0048 accuracy:1.0000
step:90 loss:0.0057 accuracy:1.0000
step:100 loss:0.0045 accuracy:1.0000


## 混淆矩阵

In [22]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def predictAll(test_X, batch_size=100):
    predict_value_list = []
    for i in range(0, len(test_X), batch_size):
        selected_X = test_X[i: i + batch_size]
        predict_value = session.run(predict_Y, {X_holder:selected_X})
        predict_value_list.extend(predict_value)
    return np.array(predict_value_list)

Y = predictAll(test_X)
y = np.argmax(Y, axis=1)
predict_label_list = labelEncoder.inverse_transform(y)
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,beierzi,beilaogong,beilaopo,beinver
beierzi,94,0,0,0
beilaogong,0,117,0,0
beilaopo,0,0,109,0
beinver,0,0,0,108


## 报告表

In [23]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def eval_model(y_true, y_pred, labels):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(y_true, y_pred)
    # 计算总体的平均Precision, Recall, f1, support
    tot_p = np.average(p, weights=s)
    tot_r = np.average(r, weights=s)
    tot_f1 = np.average(f1, weights=s)
    tot_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': labels,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [tot_p],
        u'Recall': [tot_r],
        u'F1': [tot_f1],
        u'Support': [tot_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

,Label,Precision,Recall,F1,Support
0,beierzi,1.0,1.0,1.0,94
1,beilaogong,1.0,1.0,1.0,117
2,beilaopo,1.0,1.0,1.0,109
3,beinver,1.0,1.0,1.0,108
999,总体,1.0,1.0,1.0,428
